# 03 — Analytical Modeling & KPI Definitions (SQL-Ready)

Project: SupportOps  
Phase: 3 (Modeling & Metrics)

## Purpose
This notebook locks in the reporting grain (fact table level), the dimensional model, and the KPI definitions that will be implemented in SQL (Phase 4) and Power BI (Phase 5).

### What this notebook DOES
- Defines the grain (fact table level)
- Specifies a BI-friendly star schema
- Defines a small set of operational KPIs (with SQL-ready logic)

### What this notebook does NOT do
- Heavy EDA (that was Phase 2)
- Time-based SLA/aging metrics (timestamps not present)
- Agent-level productivity (agent identifier not present)

## 1. Define the Grain (Fact Table Level)

**Grain:** 1 row = 1 support ticket.

All KPIs in this project aggregate from this grain.  
No KPI is allowed to change the grain (e.g., “per message”, “per status event”) because the dataset does not contain ticket event history.

In [1]:
import pandas as pd

RAW_PATH = "../data/raw/dataset-tickets-multi-lang-4-20k.csv"

df_raw = pd.read_csv(RAW_PATH)
df = df_raw.copy()

# Light standardization (minimal)
cat_cols = ["priority", "queue", "type", "language"]
for c in cat_cols:
    df[c] = df[c].astype(str).str.strip().str.lower()

df.shape, df.columns

((20000, 15),
 Index(['subject', 'body', 'answer', 'type', 'queue', 'priority', 'language',
        'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8'],
       dtype='object'))

## 2. Reporting Model (Star Schema)

We will structure the data for BI reporting using a star schema.

### Fact Table
**fact_tickets** (1 row per ticket)
- ticket_id (surrogate key)
- priority
- queue
- type
- language

> Note: The dataset does not provide a native `ticket_id`, so we will generate a surrogate key.

### Dimension Tables
- **dim_priority**
- **dim_queue**
- **dim_type**
- **dim_language**

### Many-to-many: Tags (Issue Classifications)
Tags appear across multiple columns (`tag_1 ... tag_8`).  
We will model tags using a bridge table:

- **dim_tag** (unique tag values)
- **bridge_ticket_tag** (ticket_id ↔ tag)

This supports accurate “top recurring issues” reporting without duplicating tickets.

In [2]:
# Create surrogate ticket id
df = df.reset_index(drop=True)
df["ticket_id"] = df.index + 1  # stable within this file

# Build tag bridge (long format)
tag_cols = [c for c in df.columns if c.startswith("tag_")]

tag_long = (
    df[["ticket_id", "language"] + tag_cols]
    .melt(id_vars=["ticket_id", "language"], value_vars=tag_cols, var_name="tag_col", value_name="tag")
    .dropna(subset=["tag"])
)

tag_long["tag"] = tag_long["tag"].astype(str).str.strip()
tag_long = tag_long[tag_long["tag"] != ""]

df[["ticket_id", "priority", "queue", "type", "language"]].head(), tag_long.head(), tag_long.shape

(   ticket_id priority             queue      type language
 0          1      low   general inquiry  incident       de
 1          2   medium  customer service   request       en
 2          3   medium  customer service   request       en
 3          4     high  customer service  incident       de
 4          5   medium  customer service   request       en,
    ticket_id language tag_col        tag
 0          1       de   tag_1      Crash
 1          2       en   tag_1   Feedback
 2          3       en   tag_1  Technical
 3          4       de   tag_1   Security
 4          5       en   tag_1   Security,
 (104597, 4))

## KPI 1 — Ticket Volume

**Definition:** Count of tickets grouped by an operational dimension (e.g., queue, priority, type, language).

**Why it matters:** Volume is the base measure for workload analysis, staffing, and operational planning.

**Operational question answered:**  
- “Where is the work landing?”

**SQL-ready logic:**  
- `COUNT(*)` grouped by dimension(s)

**Limits:**  
- No trend over time (no created/resolved timestamps).

In [3]:
kpi_ticket_volume_by_queue = (
    df.groupby("queue")["ticket_id"]
    .count()
    .reset_index(name="ticket_count")
    .sort_values("ticket_count", ascending=False)
)

kpi_ticket_volume_by_queue.head(10)

,queue,ticket_count
9,technical support,5824
5,product support,3708
1,customer service,3152
4,it support,2292
0,billing and payments,2086
6,returns and exchanges,1001
8,service outages and maintenance,764
7,sales and pre-sales,572
3,human resources,338
2,general inquiry,263


## KPI 2 — Priority Mix

**Definition:** Distribution of tickets across priority levels.

**Why it matters:** Distinguishes urgent operational load from routine requests.

**Operational question answered:**  
- “How much of our workload is truly urgent?”

**SQL-ready logic:**  
- `COUNT(*)` by priority  
- Optional: `% of total tickets` by priority

**Limits:**  
- Priority is treated as provided (no SLA timestamps to validate urgency).

In [4]:
kpi_priority_mix = df["priority"].value_counts(dropna=False)
kpi_priority_mix

priority
medium    8144
high      7801
low       4055
Name: count, dtype: int64

## KPI 3 — Queue Workload Concentration

**Definition:** Distribution of tickets across queues to identify concentration of workload.

**Why it matters:** Highlights operational bottlenecks and potential routing inefficiencies.

**Operational question answered:**  
- “Where is work accumulating?”

**SQL-ready logic:**  
- `COUNT(*)` by queue  
- Sort descending  
- Optional: cumulative share (Pareto) in later phase

**Limits:**  
- Queue is used as the best available “ownership proxy” (no agent ids).

In [8]:
kpi_queue_counts = df["queue"].value_counts()
kpi_queue_counts.head(10)

queue_counts = df["queue"].value_counts()
total = queue_counts.sum()

top_n = 3
top_n_share = (queue_counts.head(top_n).sum() / total)

queue_share = (queue_counts / total).rename("share").reset_index().rename(columns={"index":"queue"})
queue_share["cum_share"] = queue_share["share"].cumsum()

top_n_share, queue_share.head(10)

(np.float64(0.6342),
                              queue    share  cum_share
 0                technical support  0.29120    0.29120
 1                  product support  0.18540    0.47660
 2                 customer service  0.15760    0.63420
 3                       it support  0.11460    0.74880
 4             billing and payments  0.10430    0.85310
 5            returns and exchanges  0.05005    0.90315
 6  service outages and maintenance  0.03820    0.94135
 7              sales and pre-sales  0.02860    0.96995
 8                  human resources  0.01690    0.98685
 9                  general inquiry  0.01315    1.00000)

## KPI 4 — Ticket Type Mix

**Definition:** Distribution of tickets by type (incident, request, problem, change).

**Why it matters:** Reveals whether the org is mostly reacting (incidents) vs handling planned support (requests).

**Operational question answered:**  
- “What kind of work dominates support operations?”

**SQL-ready logic:**  
- `COUNT(*)` by type

**Limits:**  
- Assumes type labels are mutually exclusive as provided.

In [6]:
kpi_type_mix = df["type"].value_counts()
kpi_type_mix

type
incident    7978
request     5763
problem     4184
change      2075
Name: count, dtype: int64

## KPI 5 — Top Recurring Issues (Tags)

**Definition:** Most frequently occurring issue classifications derived from tag fields.

**Why it matters:** Identifies systematic problem areas and self-service/deflection opportunities.

**Operational question answered:**  
- “Where are we repeatedly failing — and why?”

**SQL-ready logic:**  
- Normalize tags to long format (ticket_id ↔ tag)  
- `COUNT(DISTINCT ticket_id)` by tag (or `COUNT(*)` depending on definition)

**Limits:**  
- Tags reflect classification, not lifecycle status.
- Many-to-many relationship: one ticket can have multiple tags.

In [7]:
kpi_top_tags = tag_long["tag"].value_counts().head(20)
kpi_top_tags

tag
Tech Support     8051
IT               8002
Documentation    7062
Feedback         6767
Performance      6629
Bug              6572
Technical        5804
Security         4754
Resolution       4165
Feature          3469
Guidance         2883
Product          2597
Customer         2298
Crash            1919
Outage           1898
Network          1890
Integration      1538
Sales            1527
Disruption       1361
Billing          1252
Name: count, dtype: int64

## Phase 3 Summary — Reporting Model & KPI Lock-In

### What we locked in
- **Grain:** 1 row = 1 support ticket
- **Model:** Star schema concept
  - fact_tickets + dimensions (priority, queue, type, language)
  - tags modeled as **bridge_ticket_tag** (many-to-many)

### Official KPI set supported by this dataset
1. Ticket Volume
2. Priority Mix
3. Queue Workload Concentration
4. Ticket Type Mix
5. Top Recurring Issues (Tags)

### Explicit limitations (we will NOT claim these)
- No time-based SLA / aging / time-to-resolution metrics (timestamps not present)
- No agent productivity metrics (agent ids not present)
- No backlog trend (no open/closed or created/resolved timestamps)

### Why this matters
This notebook ensures KPIs are defined once and can be implemented consistently in **SQL (Phase 4)** and **Power BI (Phase 5)** without re-interpreting logic inside dashboards.